<a href="https://colab.research.google.com/github/jimenabarrerau/Act7/blob/main/Act7_A01706474.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

In [15]:
df=pd.read_csv('cuentas_credicel.csv', encoding = 'latin')
df.sample(10)

<ipython-input-15-e883e98029ae>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('cuentas_credicel.csv', encoding = 'latin')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
16063,16236,NXXSZ28T,64001,22/01/23 11:35,SAMSUNG,4G SM-A042M A04E Azul Kit,13S,2689.83,1015.0,0.0,...,37.73,3500,12,89602,39.0,ALTAMIRA,TAMP,Papantla,Veracruz,RAEL830522HTSNSS05
12356,12509,3291UXL3,52344,12/12/22 13:35,SAMSUNG,4G A035M GXY A03 3+32GB NEGRO KIT,26S,2982.33,660.0,0.0,...,22.13,4000,18,93419,44.0,PAPANTLA,VER,Papantla,Veracruz,OAGE780510HVZLRD05
20247,20432,G3O883K6,84937,17/03/23 17:54,Samsung,A13,26S,4999.00,1315.0,0.0,...,26.31,4000,5,40000,45.0,IGUALA DE LA INDEPENDENCIA,GRO,Iguala De La Independencia,Guerrero,CAPH770901MGRHLY06
10640,10792,B3T277CC,37871,16/11/22 20:25,Motorola,MOTO G41,39S,4749.00,997.0,250.0,...,20.99,6000,22,97345,24.0,MERIDA,YUC,Chicxulub,Yucatan,RERJ980212HMCYMN01
6527,6662,HYXUNVI6,24858,06/08/22 12:08,Samsung,A03s,39S,4980.00,804.0,0.0,...,16.14,9000,37,NaN,27.0,SAN MARTIN TEXMELUCAN,PUE.,San Martin Texmelucan,Puebla,DESN941219MTLLLM09
16194,16367,9EZ080IC,66617,23/01/23 16:33,SAMSUNG,4G A035M GXY A03 3+32GB NEGRO KIT,26S,2923.83,705.0,0.0,...,24.11,4000,12,93546,20.0,PAPANTLA,VER,Papantla,Veracruz,HEHR020610HVZRRMA8
21761,21966,5ZOYY0KO,92056,06/04/23 11:42,Motorola,Moto E22i,39S,3299.00,852.0,0.0,...,25.83,5500,2,73390,35.0,HERMENEGILDO GALEANA,PUE,Zacatlan,Puebla,VAVD870826MPLZZL01
2291,2424,X7AVRJQ2,10967,25/05/22 14:01,Motorola,XT2167-1 MOTO G41,39S,5849.00,1170.0,0.0,...,20.00,6000,47,NaN,27.0,MERIDA,YUC.,Conkal,Yucatan,CECB940724MYNMXR09
1657,1665,BPYAQKA3,8596,08/05/22 17:23,Motorola,XT2143-1 EDGE 20 5G,13S,10999.00,4999.0,0.0,...,45.45,6000,13,NaN,25.0,OMETEPEC,GRO.,San Luis Acatlan,Guerrero,GULN961130MGRLLY07
8937,9083,E0A1G6R7,36201,13/10/22 15:09,HISENSE,E20S 2+32 GB AZUL B,26S,2572.83,409.0,0.0,...,15.90,8500,26,93080,52.0,MECATLAN,VER,Mecatlan,Veracruz,FAGJ700319MVZRRS02


In [16]:
df['limite_credito'] = pd.to_numeric(df['limite_credito'], errors='coerce')
mediana = df['limite_credito'].median()
df['limite_credito'].fillna(mediana, inplace=True)
df['limite_credito'] = df['limite_credito'].astype(int)
df['limite_credito']

0         5000
1         6000
2            0
3         7000
4         7000
         ...  
22730    10000
22731     5000
22732     4750
22733     7000
22734     4750
Name: limite_credito, Length: 22735, dtype: int64

In [17]:
#Para rellenar nulos empleo leyenda "dato no disponible"
df.fillna("Dato no disponible", inplace=True)

In [18]:
nulos= df.isnull().sum()
nulos

folio                 0
tag                   0
folio_solicitud       0
fecha                 0
marca                 0
modelo                0
plazo                 0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
agente_venta          0
dis_venta             0
status                0
fraude                0
empresa               0
inversion             0
pagos_realizados      0
reautorizacion        0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
puntos                0
riesgo                0
score_buro            0
razones_buro          0
porc_eng              0
limite_credito        0
semana_actual         0
cp_cliente            0
edad_cliente          0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

In [19]:
#Tratamiento de outliers
for i in df.columns:
  if i != 'folio' and i != 'folio_solicitud':
    if df[i].dtypes == 'int64' or df[i].dtypes == 'float64':
      q1 = df[i].quantile(0.25)
      q3 = df[i].quantile(0.75)
      iqr = q3 - q1
      li = q1 - 1.5 * iqr
      ls = q3 + 1.5 * iqr
      df = df[(df[i] >= li) & (df[i] <= ls)]

In [20]:
df = df.drop(columns = [i for i in df.columns if df[i].dtypes == 'object'])

In [21]:
res = pd.DataFrame(columns=['Variable', 'Precisión', 'Exactitud', 'Puntaje F1'])
df1 = df.copy()
q50 = df['score_buro'].quantile(0.5)

def clasificacion(valor):
    if valor <= q50:
        return 'bajo'
    else:
        return 'alto'

df1['score_buro'] = df1['score_buro'].apply(clasificacion)

In [22]:
y = df1['score_buro']
X = df1[['precio',
        'costo_total',
        'enganche',
        'monto_financiado',
        'monto_accesorios',
        'pagos_realizados',
        'reautorizacion',
        'limite_credito',
        'descuento']]

In [23]:
#Modelo
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state =7)
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [26]:
y_pred = algoritmo.predict(X_test)

matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="binary", pos_label="alto")
exactitud = accuracy_score(y_test, y_pred)
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="alto")

var = 'Todas'

res = res.append({'Variable': var,
                                  'Precisión': precision,
                                  'Exactitud': exactitud,
                                  'Puntaje F1': puntajef1},
                                 ignore_index=True)

print('Matriz de Confusión:',matriz)
print('\nPrecisión del modelo:',precision)
print('\nExactitud del modelo:')
print('\nPuntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión: [[716 237]
 [120 835]]

Precisión del modelo:
0.8564593301435407

Exactitud del modelo:
0.8128930817610063

Puntaje F1 del modelo:
0.8004471771939632


<ipython-input-26-b1251333bd07>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append({'Variable': var,
